<a href="https://colab.research.google.com/github/Shreeshambav/DeepLearning_training/blob/main/Transformer_Sentimentanalysis_Huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Transformers installation
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.0 MB/s eta 0:00:00


In [3]:
# Getting started on a task with a pipeline
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
classifier('We are very happy to show you the 🤗 Transformers library.')

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

In [5]:
classifier('The pizza is not that great but the crust is awesome.')

[{'label': 'POSITIVE', 'score': 0.9998461008071899}]

In [6]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.",
           "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


In [7]:
# The model downloaded for this pipeline is called "distilbert-base-uncased-finetuned-sst-2-english - DistilBERT architecture
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

In [8]:
classifier("Esperamos que no lo odie.")

[{'label': '3 stars', 'score': 0.3368820548057556}]

In [11]:
classifier("Nous espérons que vous ne le détestez pas.")

[{'label': '5 stars', 'score': 0.49765723943710327}]

In [9]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [10]:
# To do it in local
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
# This model only exists in PyTorch, so we use the `from_pt` flag to import that model in TensorFlow.
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [12]:
classifier("I am a good boy")

[{'label': '4 stars', 'score': 0.4229269027709961}]

In [13]:
# Under the hood: pretrained models - There are many tokenizers
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [14]:
# Using the tokenizer
inputs = tokenizer("We are very happy to show you the 🤗 Transformers library.")

In [15]:
print(inputs)

{'input_ids': [101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [16]:
# You can pass a list of sentences directly to your tokenizer. If your goal is to send them through your model as a batch, you probably want to pad them all to the same length, truncate them to the maximum length the model can accept and get tensors back. You can specify all of that to the tokenizer:
tf_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="tf"
)

In [17]:
for key, value in tf_batch.items():
    print(f"{key}: {value.numpy().tolist()}")

input_ids: [[101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], [101, 2057, 3246, 2017, 2123, 1005, 1056, 5223, 2009, 1012, 102, 0, 0, 0]]
attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]


In [18]:
# Using the model - Once your input has been preprocessed by the tokenizer, you can send it directly to the model.
tf_outputs = tf_model(tf_batch)

In [19]:
# In 🤗 Transformers, all outputs are tuples
print(tf_outputs)

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-4.0832963 ,  4.336415  ],
       [ 0.08181015, -0.04178584]], dtype=float32)>, hidden_states=None, attentions=None)


In [20]:
# Let's apply the SoftMax activation to get predictions.
import tensorflow as tf
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)

In [21]:
print(tf_predictions)

tf.Tensor(
[[2.2042972e-04 9.9977952e-01]
 [5.3085971e-01 4.6914026e-01]], shape=(2, 2), dtype=float32)


In [22]:
# If you have labels, you can provide them to the model, it will return a tuple with the loss and the final activations.
import tensorflow as tf
tf_outputs = tf_model(tf_batch, labels = tf.constant([1, 0]))

In [25]:
# Models are standard torch.nn.Module or tf.keras.Model so you can use them in your usual training loop. 🤗 Transformers also provides a Trainer (or TFTrainer if you are using TensorFlow) class to help with your training (taking care of things such as distributed training, mixed precision, etc.). See the training tutorial for more details.
# Once your model is fine-tuned, you can save it with its tokenizer in the following way:


In [29]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Save the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer.save_pretrained("E:\\Raj\\Book-working\\10-Python\\Program\\Alpha_Optimum_Python\\Projects\\NLTK\\project_nlp_translation\\EDA\\Token")

# Save the model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
model.save_pretrained("E:\\Raj\\Book-working\\10-Python\\Program\\Alpha_Optimum_Python\\Projects\\NLTK\\project_nlp_translation\\EDA\\Model\\EDA_toktransHF")




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
# You can then load this model back using the AutoModel.from_pretrained method by passing the directory name instead of the model name. One cool feature of 🤗 Transformers is that you can easily switch between PyTorch and TensorFlow: any model saved as before can be loaded back either in PyTorch or TensorFlow. If you are loading a saved PyTorch model in a TensorFlow model, use TFAutoModel.from_pretrained like this:
tokenizer = AutoTokenizer.from_pretrained("E:\\Raj\\Book-working\\10-Python\\Program\\Alpha_Optimum_Python\\Projects\\NLTK\\project_nlp_translation\\EDA\\EDA_token")
model = TFAutoModel.from_pretrained("E:\\Raj\\Book-working\\10-Python\\Program\\Alpha_Optimum_Python\\Projects\\NLTK\\project_nlp_translation\\EDA\\EDA_toktransHF", from_pt=True)

NameError: ignored

In [27]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("E:\\Raj\\Book-working\\10-Python\\Program\\Alpha_Optimum_Python\\Projects\\NLTK\\project_nlp_translation\\EDA\\EDA_token")

# Load the model
model = TFAutoModelForSequenceClassification.from_pretrained("E:\\Raj\\Book-working\\10-Python\\Program\\Alpha_Optimum_Python\\Projects\\NLTK\\project_nlp_translation\\EDA\\EDA_toktransHF", from_pt=True)


UnpicklingError: ignored

In [ ]:
# Lastly, you can also ask the model to return all hidden states and all attention weights if you need them:
tf_outputs = tf_model(tf_batch, output_hidden_states=True, output_attentions=True)
all_hidden_states, all_attentions = tf_outputs[-2:]

In [ ]:
# Accessing the code
# The AutoModel and AutoTokenizer classes are just shortcuts that will automatically work with any pretrained model. Behind the scenes, the library has one model class per combination of architecture plus class, so the code is easy to access and tweak if you need to.
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFDistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

In [ ]:
# Customising the model
# If you want to change how the model itself is built, you can define your custom configuration class. Each architecture comes with its own relevant configuration (in the case of DistilBERT, DistilBertConfig) which allows you to specify any of the hidden dimension, dropout rate, etc. If you do core modifications, like changing the hidden size, you won't be able to use a pretrained model anymore and will need to train from scratch. You would then instantiate the model directly from this configuration.
# Here we use the predefined vocabulary of DistilBERT (hence load the tokenizer with the DistilBertTokenizer.from_pretrained method) and initialize the model from scratch (hence instantiate the model from the configuration instead of using the DistilBertForSequenceClassification.from_pretrained method).

from transformers import DistilBertConfig, DistilBertTokenizer, TFDistilBertForSequenceClassification
config = DistilBertConfig(n_heads=8, dim=512, hidden_dim=4*512)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification(config)

In [ ]:
# For something that only changes the head of the model (for instance, the number of labels), you can still use a pretrained model for the body.
from transformers import DistilBertConfig, DistilBertTokenizer, TFDistilBertForSequenceClassification
model_name = "distilbert-base-uncased"
model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=10)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)